
# install dependencies
! pip install -U pymoo

In [2]:
! pip install -U pymoo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 KB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 KB 19.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for grapheme: filename=grapheme-0.6.0-py3-none-any.whl size=210096 sha256=ba13102054eb922a613ce459c87eaf967b4f188c998b5c25233c1eb5628bb7b6
  Stored in directory: /root/.cache/pip/wheels/2d/08/6b/126ea9009f7482fd53a78d0db2ece5aca70af8f4a30445386b
Successfully built grapheme


In [4]:
from numpy.core.memmap import uint8
import numpy as np

class AdaptiveWeight:
    def __init__(self, N, Max_iter, F, K):
        self.Ns = np.zeros(( Max_iter+1, N))
        self.As = np.zeros(( Max_iter+1, N))
        self.Is = np.zeros(( Max_iter+1, N))
        self.Iter = 0
        self.Max_iter = Max_iter
        self.N_track = N
        self.pobWeights = np.ones((N,))/N
        self.weights = np.ones((N,))
        self.Ns[0] = np.ones((N,))
        self.As[0] = np.ones((N,))
        self.Is[0] = np.ones((N,))
        self.F = F
        self.K = K
        self.listIndexs = list(np.arange(N, dtype=np.uint8))

    def random_tracks(self, size):
        tracksIds = np.random.choice(self.listIndexs, size, p=self.pobWeights)
        return tracksIds

    def update(self, tracks, objects, bestI):
        self.Iter += 1
        self.Is[self.Iter, bestI] = 1
        for i in range(self.N_track):
            indexs = np.where(tracks==i)[0]
            self.Ns[self.Iter, i] = len(indexs)
            self.As[self.Iter, i] = np.mean(objects[indexs])
            if len(indexs) == 0:
                self.As[self.Iter, i] = 0
            Ii = np.sum(self.Is[:self.Iter, i])

            new_weight = (self.F*self.Ns[self.Iter, i] + 
                              (1-self.F)*self.As[self.Iter, i] + self.K*Ii)
            self.weights[i] =  max(0.3*self.weights[i] + 0.7*new_weight, 0)
        total = np.sum(self.weights)
        #print(self.weights, total)
        for i in range(self.N_track):
            self.pobWeights[i] = self.weights[i]/total


In [6]:
class AMIS:
    def __init__(self, problem, pop_size,  max_iter, CR=0.3, CRT=0.2, F1=0.6, F2=0.4, lo=0.5):
        self.pop_size = pop_size
        self.CR = CR
        self.problem = problem
        self.max_iter = max_iter
        self.Xs = None
        self.fitnessXs = None
        self.bestX = None
        self.best2X = None
        self.CRT = CRT
        self.bestFitness = 0
        self.F1 = F1
        self.F2 = F2
        self.lo = lo
        self.data = np.zeros((3*max_iter, self.problem.n_var))
        self.out = {}
        self.initialize()
        self.N_blackboxs = 3
        self.adaptiveWeight = AdaptiveWeight(self.N_blackboxs, max_iter, 0.05, 1)

    def initialize(self):
        self.Xs = np.random.rand( self.pop_size, self.problem.n_var)
        self.fitnessXs = np.zeros((self.pop_size,))
        self.cal_fitness()
        max_index = np.argmax(self.fitnessXs)
        self.bestX = np.copy(self.Xs[max_index])
        self.best2X = np.copy(self.bestX)
        self.bestFitness = self.fitnessXs[max_index]
        
    def cal_fitness(self):
        for i in range(self.pop_size):
            self.problem._evaluate(self.Xs[i], self.out)
            self.fitnessXs[i] = self.out['F']
        
    def IB1_best(self, xs):
        N = len(xs)
        indexs = np.arange(0, self.pop_size, 1).astype(dtype=np.uint8)
        index_r1s = list(np.copy(indexs))
        index_r2s = list(np.copy(indexs))
        index_r3s = list(np.copy(indexs))
        np.random.shuffle(index_r1s)
        np.random.shuffle(index_r2s)
        np.random.shuffle(index_r3s)
        Vs = self.lo*self.Xs[index_r1s[:N]] + self.F1*(self.bestX - self.Xs[index_r1s[:N]]) +\
                                              self.F2*(self.Xs[index_r2s[:N]] - self.Xs[index_r3s[:N]])
        
        vmin = np.min(Vs)
        vmax = np.max(Vs)
        if vmin != vmax:
            Vs = (Vs-vmin)/(vmax-vmin)
        return Vs


    def IB2_best2(self, xs):
        N = len(xs)
        indexs = np.arange(0, self.pop_size, 1).astype(dtype=np.uint8)
        index_r1s = list(np.copy(indexs))
        index_r2s = list(np.copy(indexs))
        index_r3s = list(np.copy(indexs))
        np.random.shuffle(index_r1s)
        np.random.shuffle(index_r2s)
        np.random.shuffle(index_r3s)
        Vs = self.Xs[index_r1s[:N]] + self.F1*(self.bestX - self.Xs[index_r1s[:N]]) +\
                                              self.F2*(self.best2X - self.Xs[index_r1s[:N]])
        
        vmin = np.min(Vs)
        vmax = np.max(Vs)
        if vmin != vmax:
            Vs = (Vs-vmin)/(vmax-vmin)
        return Vs

    def IB3_DE(self, xs):
        N = len(xs)
        indexs = np.arange(0, self.pop_size, 1).astype(dtype=np.uint8)
        index_r1s = list(np.copy(indexs))
        index_r2s = list(np.copy(indexs))
        index_r3s = list(np.copy(indexs))
        np.random.shuffle(index_r1s)
        np.random.shuffle(index_r2s)
        np.random.shuffle(index_r3s)
        Vs = self.Xs[index_r1s[:N]] + self.F1*(self.best2X - self.Xs[index_r1s[:N]])
        
        vmin = np.min(Vs)
        vmax = np.max(Vs)
        if vmin != vmax:
            Vs = (Vs-vmin)/(vmax-vmin)
        return Vs

    def IB4_DER(self, xs):
        N = len(xs)
        indexs = np.arange(0, self.pop_size, 1).astype(dtype=np.uint8)
        index_r1s = list(np.copy(indexs))
        index_r2s = list(np.copy(indexs))
        index_r3s = list(np.copy(indexs))
        np.random.shuffle(index_r1s)
        np.random.shuffle(index_r2s)
        np.random.shuffle(index_r3s)
        Vs = self.Xs[index_r1s[:N]] + np.random.rand()*(self.best2X - self.Xs[index_r1s[:N]])
        
        vmin = np.min(Vs)
        vmax = np.max(Vs)
        if vmin != vmax:
            Vs = (Vs-vmin)/(vmax-vmin)
        return Vs

    def IB5_random(self, xs):
        N = len(xs)
        rs = np.random.rand(xs.shape[0], xs.shape[1])
        cross_select = np.random.choice(a=[0, 1], size=xs.shape, p=[self.CRT, 1-self.CRT])  
        Vs = rs
        return Vs
    
    
    def IB6_random_transit(self, xs):
        N = len(xs)
        rs = np.random.rand(xs.shape[0], xs.shape[1])
        cross_select = np.random.choice(a=[0, 1], size=xs.shape, p=[self.CRT, 1-self.CRT])  
        Vs = (1-cross_select)*xs + cross_select*rs
        return Vs
    
    def IB7_best_transit(self, xs):
        N = len(xs)
        rs = np.random.rand(xs.shape[0], xs.shape[1])
        cross_select = np.random.choice(a=[0, 1], size=xs.shape, p=[self.CRT, 1-self.CRT])  
        Vs = (1-cross_select)*xs + cross_select*self.bestX
        return Vs

    def IB8_scale(self, xs):
        N = len(xs)
        rs = np.random.rand(xs.shape[0], xs.shape[1])
        cross_select = np.random.choice(a=[0, 1], size=xs.shape, p=[self.CRT, 1-self.CRT])  
        Vs =  ((1 - cross_select)*rs)*xs +   cross_select*xs
        return Vs

    def IB9_select_another(self, xs):
        N = len(xs)
        indexs = np.arange(0, self.pop_size, 1).astype(dtype=np.uint8)
        index_r1s = list(np.copy(indexs))
        index_r2s = list(np.copy(indexs))
        np.random.shuffle(index_r1s)
        np.random.shuffle(index_r2s)
        cross_select = np.random.choice(a=[0, 1], size=xs.shape, p=[self.CRT, 1-self.CRT])  
        Vs =  ((1 - cross_select)*self.Xs[index_r1s[:N]])*xs +   cross_select*self.Xs[index_r2s[:N]]
        return Vs

    def select_blocks(self):
        self.Vs = np.copy(self.Xs)
        for i in range(self.N_blackboxs):
            indexI = list(np.where(self.currentTracks == i)[0])
            if len(indexI) == 0:
                continue
            if i == 0:
                self.Vs[indexI] = self.IB3_DE(self.Xs[indexI])
            elif i==1:
                self.Vs[indexI] = self.IB6_random_transit(self.Xs[indexI])
            elif i==2:
                self.Vs[indexI] = self.IB9_select_another(self.Xs[indexI])

            
            

    def recombination(self):
        cross_select = np.random.choice(a=[0, 1], size=self.Xs.shape, p=[self.CR, 1-self.CR])  
        self.Us = (1-cross_select)*self.Xs + cross_select*self.Vs
        
    def accept_all(self):
        fitnessUs = np.zeros((self.pop_size,))
        select_mask = np.zeros((self.pop_size,))
        for i in range(self.pop_size):
            self.problem._evaluate(self.Us[i], self.out)
            fitnessUs[i] = self.out['F']
            self.fitnessXs[i] = fitnessUs[i]
            self.Xs[i] = np.copy(self.Us[i])
            if fitnessUs[i] > self.bestFitness:
                self.bestFitness = fitnessUs[i]
                self.best2X = np.copy(self.bestX)
                self.bestX = np.copy(self.Us[i])

    def selection(self):
        fitnessUs = np.zeros((self.pop_size,))
        select_mask = np.zeros((self.pop_size,))
        for i in range(self.pop_size):
            self.problem._evaluate(self.Us[i], self.out)
            fitnessUs[i] = self.out['F']
            delta = fitnessUs[i] - self.fitnessXs[i]
            if (delta < 0 or 
                np.random.rand() + 5 < np.exp(-delta*100) or
                np.random.rand()  < 0.00001):
                self.fitnessXs[i] = fitnessUs[i]
                self.Xs[i] = np.copy(self.Us[i])
                if fitnessUs[i] < self.bestFitness:
                    self.bestFitness = fitnessUs[i]
                    self.best2X = np.copy(self.bestX)
                    self.bestX = np.copy(self.Us[i])



    def iterate(self):
        for it in range(self.max_iter):
            self.currentTracks = self.adaptiveWeight.random_tracks(self.pop_size)
            self.select_blocks()
            arg_max = np.argmax(self.fitnessXs)
            self.adaptiveWeight.update(self.currentTracks, self.fitnessXs, self.currentTracks[arg_max])
            self.recombination()
            self.selection()

            idx= np.argsort(self.bestX)

            print(it, round(np.mean(self.fitnessXs), 4), round(self.bestFitness, 4), self.adaptiveWeight.pobWeights)
            arg_maxs = np.argsort(self.fitnessXs)
            self.data[it*3:3*(it+1)] = np.copy(self.Xs[arg_maxs[:3]])

In [9]:
from pymoo.algorithms.soo.nonconvex.brkga import BRKGA
from sklearn.metrics import accuracy_score
from pymoo.algorithms.soo.nonconvex.de import DE
import numpy as np
from pymoo.core.problem import ElementwiseProblem
import math

class WC_EnsembleProblem(ElementwiseProblem):
    
    def __init__(self,preds, ys_c):
        self.N = len(preds)
        self.preds = preds
        self.ys_c = ys_c
        super().__init__(n_var=self.N, n_obj=1, n_constr=0, xl=0, xu=1)

    

    def _evaluate(self, xs, out, *args, **kwargs):
        w = (xs )/np.sum(xs)
        ypred_en = self.preds[0]*w[0]
        for i in range(1, len( self.preds)):
            ypred_en += (self.preds[i]*w[i])

        indexs = np.where(ypred_en>1)
        ypred_en[indexs] = 1

        ypred_max_en = np.argmax(ypred_en, axis=1)
        d = ypred_en - self.ys_c
      

        f = 1 - accuracy_score(ys, ypred_max_en)
        
        out["hash"] = hash(str(w))
        out["F"] = f
        out["pheno"] = {"weights": w}
     